# Objective

The objective of this notebook is to understand the metrics in the sample_tableau_data file of module-progress


* Improve the code by referecing with IDs wherever necessary
* Transfer the code to a dash app
* Remove/Add the labels from the plots where necessary
* Improve dashboard aesthetics

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from collections import defaultdict

from datetime import *
import datetime
pio.renderers.default='iframe'

In [2]:
data = pd.read_csv('../data/SAMPLE_module_data.csv')

In [3]:
data.head()

,Unnamed: 0,completed_at,course_id,module_id,items_count,module_name,module_position,state,unlock_at,student_id,student_name,items_id,items_title,items_position,items_indent,items_type,items_module_id,item_cp_req_type,item_cp_req_completed,course_name
0,0,2019-06-25 22:29,2591,8135,3,Module 1: Course Introduction,1,completed,NaN,1001,student01,85224,Introduction Text,1,0,Page,8135,must_mark_done,True,Canvas-Sandbox-Course
1,1,2019-06-25 22:29,2591,8135,3,Module 1: Course Introduction,1,completed,NaN,1001,student01,85225,Important Info Moving Foreward,2,0,Page,8135,must_mark_done,True,Canvas-Sandbox-Course
2,2,2019-06-25 22:29,2591,8135,3,Module 1: Course Introduction,1,completed,NaN,1001,student01,85262,Quiz 1 - Past Experience,3,1,Quiz,8135,must_submit,True,Canvas-Sandbox-Course
3,3,NaN,2591,8136,4,Module 2: How to Design Data,2,started,NaN,1001,student01,85263,Designing Data,1,0,Page,8136,must_view,False,Canvas-Sandbox-Course
4,4,NaN,2591,8136,4,Module 2: How to Design Data,2,started,NaN,1001,student01,85228,Talk About It.,2,0,Discussion,8136,must_contribute,True,Canvas-Sandbox-Course


In [4]:
data.columns

Index(['Unnamed: 0', 'completed_at', 'course_id', 'module_id', 'items_count',
       'module_name', 'module_position', 'state', 'unlock_at', 'student_id',
       'student_name', 'items_id', 'items_title', 'items_position',
       'items_indent', 'items_type', 'items_module_id', 'item_cp_req_type',
       'item_cp_req_completed', 'course_name'],
      dtype='object')

In [5]:
categorical_cols = ['course_id',
                   'module_id',
                   'module_name',
                   'state',
                   'student_id',
                   'student_name',
                   'items_id',
                   'items_title',
                   'items_type',
                   'items_module_id',
                   'item_cp_req_type',
                   'item_cp_req_completed',
                   'course_name']

In [6]:
# convert the timestamp to datetime format
# fix the column data types
data['completed_at'] = pd.to_datetime(data['completed_at'])
for col in categorical_cols:
    data[col] = data[col].astype('category')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0             420 non-null    int64         
 1   completed_at           103 non-null    datetime64[ns]
 2   course_id              420 non-null    category      
 3   module_id              420 non-null    category      
 4   items_count            420 non-null    int64         
 5   module_name            420 non-null    category      
 6   module_position        420 non-null    int64         
 7   state                  420 non-null    category      
 8   unlock_at              0 non-null      float64       
 9   student_id             420 non-null    category      
 10  student_name           420 non-null    category      
 11  items_id               420 non-null    category      
 12  items_title            420 non-null    category      
 13  items

In [8]:
# Drop the unwanted columns
data.drop(['Unnamed: 0'], axis = 1, inplace = True)

## Demo Dashboard

### Tab - Student Module Completion over time

In [9]:
modules = list(data.module_name.unique())
for module in modules:
    # Print Module Status

    # number of student id related to Module
    data_module = data[(data.module_name == module)]
    data_module['student_id'].unique().size

    # number if students that have completed the Module
    data_module[data_module.state == 'completed']['student_id'].unique().size

    # number if students that have unlocked the Module 1
    data_module[data_module.state == 'unlocked']['student_id'].unique().size

    # number if students that have started the Module 1
    data_module[data_module.state == 'started']['student_id'].unique().size


    # display
    print(f"Total number of students engaged (start/completed) {module}: {data_module['student_id'].unique().size}")
    print(f"Number of students completed {module}: {data_module[data_module.state == 'completed']['student_id'].unique().size}")

    print(f"Percentage of completion: {round(data_module[data_module.state == 'completed']['student_id'].unique().size * 100 / data_module['student_id'].unique().size, 1)}%")
    print(f"Percentage of started : {round(data_module[data_module.state == 'started']['student_id'].unique().size * 100 / data_module['student_id'].unique().size, 1)}%")
    print(f"Percentage of unlocked : {round(data_module[data_module.state == 'unlocked']['student_id'].unique().size * 100 / data_module['student_id'].unique().size, 1)}%")
    print("="*100)



Total number of students engaged (start/completed) Module 1: Course Introduction: 21
Number of students completed Module 1: Course Introduction: 18
Percentage of completion: 85.7%
Percentage of started : 14.3%
Percentage of unlocked : 0.0%
Total number of students engaged (start/completed) Module 2: How to Design Data: 21
Number of students completed Module 2: How to Design Data: 4
Percentage of completion: 19.0%
Percentage of started : 66.7%
Percentage of unlocked : 14.3%
Total number of students engaged (start/completed) Module 3: How to Design Functions: 21
Number of students completed Module 3: How to Design Functions: 5
Percentage of completion: 23.8%
Percentage of started : 28.6%
Percentage of unlocked : 47.6%
Total number of students engaged (start/completed) Module 4: Intro to Object Orientation: 21
Number of students completed Module 4: Intro to Object Orientation: 3
Percentage of completion: 14.3%
Percentage of started : 14.3%
Percentage of unlocked : 71.4%
Total number of st

In [10]:
## Functions

def get_completed_percentage(df, module,state):
    '''
    returns the state percetage of module in df
    '''
    
    df_module = df[df.module_name == module]
    total_module_students = df_module.student_id.unique().size
    percentage = df_module[df_module.state==state].student_id.unique().size / total_module_students 
    return percentage

In [11]:
## To build a dashboard display with horizontal bars like in Tableau

total_students = data.student_id.unique().size

result = {}
for module in modules:
    result[str(module)] = [round(get_completed_percentage(data, module, 'unlocked')*100, 1), 
                           round(get_completed_percentage(data, module, 'started')*100, 1),
                           round(get_completed_percentage(data, module, 'completed')*100, 1)]

In [12]:
result

{'Module 1: Course Introduction': [0.0, 14.3, 85.7],
 'Module 2: How to Design Data': [14.3, 66.7, 19.0],
 'Module 3: How to Design Functions': [47.6, 28.6, 23.8],
 'Module 4: Intro to Object Orientation': [71.4, 14.3, 14.3],
 'Module 5: Data Hierarchies': [71.4, 14.3, 14.3],
 'Module 6: Designing Software': [66.7, 33.3, 0.0]}

In [13]:
# convert this to a dataframe 
df = pd.DataFrame(result, index = ['unlocked', 'started', 'completed']).T.reset_index().rename(columns={'index': 'Module'})
df.head()

,Module,unlocked,started,completed
0,Module 1: Course Introduction,0.0,14.3,85.7
1,Module 2: How to Design Data,14.3,66.7,19.0
2,Module 3: How to Design Functions,47.6,28.6,23.8
3,Module 4: Intro to Object Orientation,71.4,14.3,14.3
4,Module 5: Data Hierarchies,71.4,14.3,14.3


* Dictionary can be used to store the module names, and underneath it another dictionary to store the number of students in each state

### Tab - Module Completion Chart

In [14]:
# Melt the DataFrame to convert columns to rows
melted_df = pd.melt(df, id_vars='Module', value_vars=['unlocked', 'started', 'completed'],
                    var_name='Status', value_name='Percentage Completion')


In [15]:
melted_df

,Module,Status,Percentage Completion
0,Module 1: Course Introduction,unlocked,0.0
1,Module 2: How to Design Data,unlocked,14.3
2,Module 3: How to Design Functions,unlocked,47.6
3,Module 4: Intro to Object Orientation,unlocked,71.4
4,Module 5: Data Hierarchies,unlocked,71.4
5,Module 6: Designing Software,unlocked,66.7
6,Module 1: Course Introduction,started,14.3
7,Module 2: How to Design Data,started,66.7
8,Module 3: How to Design Functions,started,28.6
9,Module 4: Intro to Object Orientation,started,14.3


In [16]:
# Create a horizontal bar chart using Plotly
fig = px.bar(melted_df, y='Module', x='Percentage Completion', color='Status', orientation='h',
             labels={'Percentage Completion': 'Percentage Completion (%)'},
             title='Percentage Completion by Students for Each Module',
            category_orders={'Module': sorted(melted_df['Module'].unique())})

fig.show()

### Tab - Module Completion Over Time

In [17]:
data[(data.state == 'completed') & (data.module_name == 'Module 6: Designing Software')].size

0

In [18]:
def get_completed_percentage_date(df, module, date):
    '''
    returns specifically the completed percetage of module in df till a specified date
    
    Inputs
    ------
    df: dataframe
    module: str, name of the module
    date: datetime.date, date till which the completion percentage of each module is desired
    
    Returns
    -------
    percentage, float, percentage value
    
    '''
    
    # Convert the date to datetime with time component set to midnight
    datetime_date = datetime.datetime.combine(date, datetime.datetime.min.time())
    
    df_module = df[df.module_name == module]
    total_module_students = df_module.student_id.unique().size
    
    # if there is not a sigle row with completion date then we get an datetime error since blanks are not compared to date 
    # to get around this edge case, return 0 
    if  df_module[df_module.state == 'completed'].size == 0:
        return 0.0
    
    percentage = df_module[(df_module.state=='completed') & (df_module['completed_at'].dt.date <= date)].student_id.unique().size / total_module_students 
    
    return percentage

In [19]:
# for every timestamp by date, for every module we need to compute the percentage completion for that module
# melted dataframe will have the columns: timestamp (date), module, status (optional), percentage completion

In [20]:
# fix how to remove NaT
timestamps = data['completed_at'].dt.date.unique()
timestamps = [x for x in timestamps if type(x) != pd._libs.tslibs.nattype.NaTType ]

In [21]:
timestamps

[datetime.date(2019, 6, 25),
 datetime.date(2019, 6, 26),
 datetime.date(2019, 7, 9),
 datetime.date(2019, 6, 27)]

In [22]:
## To build a dashboard lineplot with time on the x axis and the percentage completion on y axis
# for each module

result_time = pd.DataFrame(columns = ['Date', 'Module', 'Percentage Completion'])
for module in modules:
    timestamps = data[data.module_name==module]['completed_at'].dt.date.unique()
    timestamps = [x for x in timestamps if type(x) != pd._libs.tslibs.nattype.NaTType ]
    for date in timestamps:
        #print(module)
        value = round(get_completed_percentage_date(data, module, date)*100, 1)
        
        new_df = pd.DataFrame([[date, module, value]], columns = ['Date', 'Module', 'Percentage Completion'])
        
        result_time = pd.concat([result_time, new_df], ignore_index = True)

In [23]:
result_time

,Date,Module,Percentage Completion
0,2019-06-25,Module 1: Course Introduction,19.0
1,2019-06-26,Module 1: Course Introduction,57.1
2,2019-06-27,Module 1: Course Introduction,85.7
3,2019-07-09,Module 2: How to Design Data,19.0
4,2019-06-26,Module 3: How to Design Functions,14.3
5,2019-06-27,Module 3: How to Design Functions,23.8
6,2019-06-26,Module 4: Intro to Object Orientation,4.8
7,2019-06-27,Module 4: Intro to Object Orientation,14.3
8,2019-06-26,Module 5: Data Hierarchies,4.8
9,2019-06-27,Module 5: Data Hierarchies,14.3


In [24]:
# Plotting the lineplot
fig2 = go.Figure()
for module, group in result_time.groupby('Module'):
    sorted_group = group.sort_values('Date')
    
    if len(sorted_group) == 1:
        fig2.add_trace(go.Scatter(x=sorted_group['Date'], y=sorted_group['Percentage Completion'], mode='markers', name=module))
        
    else:
        fig2.add_trace(go.Scatter(x=sorted_group['Date'], y=sorted_group['Percentage Completion'], mode='lines', name=module))

In [25]:
fig2.update_layout(
    title='Percentage Completion by Module',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Percentage')
)

# Set custom start and end dates for the x-axis
start_date = '2019-06-20'
end_date = '2019-07-20'
fig2.update_xaxes(range=[start_date, end_date])

# Specify custom spacing between dates on the x-axis
date_spacing = 'D7'  # Weekly spacing, adjust as per your requirement
fig2.update_xaxes(dtick=date_spacing)

fig2.show()

## Student Completion Per Item

In [26]:
data.head()

,completed_at,course_id,module_id,items_count,module_name,module_position,state,unlock_at,student_id,student_name,items_id,items_title,items_position,items_indent,items_type,items_module_id,item_cp_req_type,item_cp_req_completed,course_name
0,2019-06-25 22:29:00,2591,8135,3,Module 1: Course Introduction,1,completed,NaN,1001,student01,85224,Introduction Text,1,0,Page,8135,must_mark_done,True,Canvas-Sandbox-Course
1,2019-06-25 22:29:00,2591,8135,3,Module 1: Course Introduction,1,completed,NaN,1001,student01,85225,Important Info Moving Foreward,2,0,Page,8135,must_mark_done,True,Canvas-Sandbox-Course
2,2019-06-25 22:29:00,2591,8135,3,Module 1: Course Introduction,1,completed,NaN,1001,student01,85262,Quiz 1 - Past Experience,3,1,Quiz,8135,must_submit,True,Canvas-Sandbox-Course
3,NaT,2591,8136,4,Module 2: How to Design Data,2,started,NaN,1001,student01,85263,Designing Data,1,0,Page,8136,must_view,False,Canvas-Sandbox-Course
4,NaT,2591,8136,4,Module 2: How to Design Data,2,started,NaN,1001,student01,85228,Talk About It.,2,0,Discussion,8136,must_contribute,True,Canvas-Sandbox-Course


In [27]:
modules

['Module 1: Course Introduction',
 'Module 2: How to Design Data',
 'Module 3: How to Design Functions',
 'Module 4: Intro to Object Orientation',
 'Module 5: Data Hierarchies',
 'Module 6: Designing Software']

In [28]:
# Creating a dictionary of items per module
item_dict = defaultdict()

for module in modules:
    item_dict[module] = list(data[data.module_name == module].items_title.unique())

In [29]:
# Checking the items in a dictionary
item_dict.get('Module 1: Course Introduction')

['Introduction Text',
 'Important Info Moving Foreward',
 'Quiz 1 - Past Experience']

In [30]:
# total student is give by 'total_students'

# result dataframe
student_completion_per_item = pd.DataFrame(columns = ['Module', 'Item', 'Item Percentage Completion', 'Item Position'])


# Computing the percentage completion in each item of a module
for module in modules:
    # number of student id related to Module
    data_module = data[(data.module_name == module)]
    

    for i, item in enumerate(item_dict.get(module)):
        data_module_item = data_module[data_module.items_title == item]
        #print(data.head())
        
        item_percent_completion = round((data_module_item[data_module_item['item_cp_req_completed'] == True]['student_id'].unique().size)*100/total_students, 0)
        new_df = pd.DataFrame([[module, item, item_percent_completion, i+1]], columns = ['Module', 'Item', 'Item Percentage Completion', 'Item Position'])
        
        student_completion_per_item = pd.concat([student_completion_per_item, new_df], ignore_index = True)

In [31]:
student_completion_per_item.head()

,Module,Item,Item Percentage Completion,Item Position
0,Module 1: Course Introduction,Introduction Text,100.0,1
1,Module 1: Course Introduction,Important Info Moving Foreward,100.0,2
2,Module 1: Course Introduction,Quiz 1 - Past Experience,86.0,3
3,Module 2: How to Design Data,Designing Data,62.0,1
4,Module 2: How to Design Data,Talk About It.,48.0,2


In [45]:
# Plotting

# Group the DataFrame by 'module'
grouped_df = student_completion_per_item.groupby('Module')


# Create subplots with one subplot per module
fig3 = make_subplots(rows=len(grouped_df), cols=1, shared_xaxes=True, vertical_spacing=0.01)

# Define custom colors for the bars
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

# Iterate over each module group
for i, (module, group) in enumerate(grouped_df):
    # Commenting out since the items should not be sorted
    # Items need to appear in the same order as in the module
    # sorted_group = group.sort_values('Item Percentage Completion', ascending=True)
    
    # Create a horizontal bar chart for the module
    fig3.add_trace(go.Bar(y=group['Item'][::-1],
                          x=group['Item Percentage Completion'][::-1],
                          orientation='h',
                          name=module,
                          marker=dict(opacity=0.8),
                          text=group['Item Position'][::-1],
                          hovertemplate="Item Position: %{text}<br>Item Title: %{y}<br>Completion:%{x}%<extra></extra>"),
                   row=i+1,
                   col=1)

# Update the layout of the figure
fig3.update_layout(height=150*len(grouped_df), title='Percentage Completion by Item for Each Module',
                  xaxis_title='Percentage Completion', yaxis_title='Item')


# Show the figure
fig3.show()

In [33]:
student_completion_per_item

,Module,Item,Item Percentage Completion,Item Position
0,Module 1: Course Introduction,Introduction Text,100.0,1
1,Module 1: Course Introduction,Important Info Moving Foreward,100.0,2
2,Module 1: Course Introduction,Quiz 1 - Past Experience,86.0,3
3,Module 2: How to Design Data,Designing Data,62.0,1
4,Module 2: How to Design Data,Talk About It.,48.0,2
5,Module 2: How to Design Data,a1 - Data Assignment,67.0,3
6,Module 2: How to Design Data,Generic Quiz,52.0,4
7,Module 3: How to Design Functions,Designing Functions,33.0,1
8,Module 3: How to Design Functions,This is a pdf.pdf,43.0,2
9,Module 3: How to Design Functions,Another Quiz,24.0,3
